In [5]:
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ConformerForCTC

In [2]:
model_path = 'model/checkpoint-1781 lb 0.01351'
processor = Wav2Vec2Processor.from_pretrained(model_path)
model = Wav2Vec2ConformerForCTC.from_pretrained(model_path).to('cuda')
model.eval()

C:\Program Files\Python39\lib\site-packages\transformers\modeling_utils.py:433: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:


Wav2Vec2ConformerForCTC(
  (wav2vec2_conformer): Wav2Vec2ConformerModel(
    (feature_extractor): Wav2Vec2ConformerFeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2ConformerLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2ConformerLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2ConformerLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2ConformerFeatureProjection(
      (layer_n

In [3]:
model(torch.randn(1, 16000*10).cuda()).logits.shape

torch.Size([1, 499, 32])

In [6]:
x = torch.randn(1, 16000*10).cuda()
os.makedirs('onnx_model', exist_ok=True)
torch.onnx.export(model, x, 'onnx_model/wav2vec2-conformer.onnx',  # will produce many small files due to 2GB limit
                  export_params=True,
                  verbose=True,
                  opset_version=18,  # TensorRT 8.6 supports up to 17, Torch 2.0.1 supports up to 18
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes={'input' : {1 : 'audio_len'}, 'output' : {1 : 'text_len'}},
                  keep_initializers_as_inputs=False)

C:\Program Files\Python39\lib\site-packages\transformers\models\wav2vec2_conformer\modeling_wav2vec2_conformer.py:430: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.pe.size(1) >= x.size(1) * 2 - 1:


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

